In [57]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.metrics import r2_score,mean_squared_error
from scipy.stats.mstats import winsorize
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.metrics import mean_squared_log_error,mean_absolute_error,r2_score
from sklearn.model_selection import train_test_split


# Feature overview:

In [2]:
pd.read_excel('Data_Dictionary.xlsx')

,Variable,Description
0,id,Unique ID for every Customer
1,account_type,Account Type – current or saving
2,gender,Gender of customer
3,age,Age of customer
4,region_code,Code assigned to region of residence (has order)
5,cc_cons_apr,Credit card spend in April
6,dc_cons_apr,Debit card spend in April
7,cc_cons_may,Credit card spend in May
8,dc_cons_may,Debit card spend in May
9,cc_cons_jun,Credit card spend in June


# Data Loading And Cleaning

In this task, we'll load the dataframe in pandas, drop the unnecessary columns and display the top five rows of the dataset.


In [5]:
data_train=pd.read_csv('train.csv').set_index('ID')

In [11]:
data_train.head()

,account_type,gender,age,region_code,dc_cons_apr,dc_cons_may,dc_cons_jun,cc_count_apr,cc_count_may,cc_count_jun,...,credit_count_may,debit_count_may,max_credit_amount_may,debit_amount_jun,credit_amount_jun,credit_count_jun,debit_count_jun,max_credit_amount_jun,loan_enq,cc_cons
ID,,,,,,,,,,,,,,,,,,,,,
12554,saving,F,36,336.0,10740.542453,26271.575951,12332.393444,4.0,6.0,12.0,...,2.0,1.0,110403.214364,1.114847e+06,6.868706e+05,2.0,1.0,161824.467583,N,1946.0
17645,current,M,24,529.0,15400.274698,16346.513708,13308.843760,14.0,10.0,31.0,...,2.0,17.0,336653.547124,4.010666e+05,2.368395e+05,10.0,31.0,530033.850635,N,66784.0
7604,current,M,34,541.0,NaN,16616.791206,8165.060330,19.0,8.0,23.0,...,4.0,22.0,359000.879855,8.944534e+05,1.896231e+06,5.0,24.0,481495.648386,Y,14469.0
1590,current,M,39,523.0,15962.461007,17848.552395,28502.691521,NaN,1.0,17.0,...,6.0,39.0,217724.536892,1.772801e+06,1.276362e+06,7.0,37.0,536714.940949,N,6494.0
16556,current,M,28,239.0,NaN,35661.995527,13474.223363,23.0,14.0,32.0,...,4.0,15.0,519029.306872,8.525740e+05,6.110580e+05,3.0,26.0,349316.588945,N,14588.0


In [12]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15000 entries, 12554 to 4547
Data columns (total 39 columns):
account_type             15000 non-null object
gender                   15000 non-null object
age                      15000 non-null int64
region_code              15000 non-null float64
dc_cons_apr              6387 non-null float64
dc_cons_may              7153 non-null float64
dc_cons_jun              5334 non-null float64
cc_count_apr             13915 non-null float64
cc_count_may             14470 non-null float64
cc_count_jun             14256 non-null float64
dc_count_apr             6387 non-null float64
dc_count_may             7153 non-null float64
dc_count_jun             7890 non-null float64
card_lim                 14991 non-null float64
personal_loan_active     1259 non-null float64
vehicle_loan_active      389 non-null float64
personal_loan_closed     1253 non-null float64
vehicle_loan_closed      711 non-null float64
investment_1             643 non-null fl

Null Value Treatment:-

In [13]:
null=pd.DataFrame(((data_train.isna().sum()/data_train.shape[0])*100),columns=['Null_value(%)'])
train_null=round(null,2)
print(train_null)

                       Null_value(%)
account_type                    0.00
gender                          0.00
age                             0.00
region_code                     0.00
dc_cons_apr                    57.42
dc_cons_may                    52.31
dc_cons_jun                    64.44
cc_count_apr                    7.23
cc_count_may                    3.53
cc_count_jun                    4.96
dc_count_apr                   57.42
dc_count_may                   52.31
dc_count_jun                   47.40
card_lim                        0.06
personal_loan_active           91.61
vehicle_loan_active            97.41
personal_loan_closed           91.65
vehicle_loan_closed            95.26
investment_1                   95.71
investment_2                   92.69
investment_3                   95.87
investment_4                   98.77
debit_amount_apr                4.29
credit_amount_apr               9.90
debit_count_apr                 5.01
credit_count_apr                5.17
m

In [14]:
# Function to identify numeric features
def numeric_features(dataset):
    numeric_col = dataset.select_dtypes(include=np.number).columns.tolist()
    return dataset[numeric_col].head()

numeric_columns = numeric_features(data_train)

# Function to identify categorical features
def categorical_features(dataset):
    categorical_col = dataset.select_dtypes(exclude=np.number).columns.tolist()
    return dataset[categorical_col].head()

categorical_columns = categorical_features(data_train)
print("Categorical Features:")
print(categorical_columns)


# Function to check the datatypes of all the columns:
def check_datatypes(dataset):
    
    return dataset.dtypes


check_datatypes(data_train)

Categorical Features:
      account_type gender loan_enq
ID                                
12554       saving      F        N
17645      current      M        N
7604       current      M        Y
1590       current      M        N
16556      current      M        N


account_type              object
gender                    object
age                        int64
region_code              float64
dc_cons_apr              float64
dc_cons_may              float64
dc_cons_jun              float64
cc_count_apr             float64
cc_count_may             float64
cc_count_jun             float64
dc_count_apr             float64
dc_count_may             float64
dc_count_jun             float64
card_lim                 float64
personal_loan_active     float64
vehicle_loan_active      float64
personal_loan_closed     float64
vehicle_loan_closed      float64
investment_1             float64
investment_2             float64
investment_3             float64
investment_4             float64
debit_amount_apr         float64
credit_amount_apr        float64
debit_count_apr          float64
credit_count_apr         float64
max_credit_amount_apr    float64
debit_amount_may         float64
credit_amount_may        float64
credit_count_may         float64
debit_coun

In [15]:
train_null=train_null[train_null>0].dropna()
null_data=data_train[train_null.index]
#categorical columns which has null value
cat_null=categorical_features(null_data).columns
# numerical columns which has null  value
num_null=numeric_features(null_data).columns

In [16]:
#cat_null imputment
data_train['loan_enq']=data_train['loan_enq'].fillna('N')

In [17]:
data_train['account_type']=data_train['account_type'].fillna(data_train['account_type'].mode()[0])
data_train['gender']=data_train['gender'].fillna(data_train['gender'].mode()[0])

In [18]:
imput_0=['personal_loan_active', 'vehicle_loan_active', 'personal_loan_closed',
       'vehicle_loan_closed', 'investment_1', 'investment_2', 'investment_3',
       'investment_4']

In [19]:
for col in imput_0:
    data_train[col]=data_train[col].fillna(0)
    

In [20]:
num=train_null[train_null<90].dropna().index

In [21]:
for col in num:
    data_train[col]=data_train[col].fillna(data_train[col].mean())

In [22]:
data_train.isna().sum()

account_type             0
gender                   0
age                      0
region_code              0
dc_cons_apr              0
dc_cons_may              0
dc_cons_jun              0
cc_count_apr             0
cc_count_may             0
cc_count_jun             0
dc_count_apr             0
dc_count_may             0
dc_count_jun             0
card_lim                 0
personal_loan_active     0
vehicle_loan_active      0
personal_loan_closed     0
vehicle_loan_closed      0
investment_1             0
investment_2             0
investment_3             0
investment_4             0
debit_amount_apr         0
credit_amount_apr        0
debit_count_apr          0
credit_count_apr         0
max_credit_amount_apr    0
debit_amount_may         0
credit_amount_may        0
credit_count_may         0
debit_count_may          0
max_credit_amount_may    0
debit_amount_jun         0
credit_amount_jun        0
credit_count_jun         0
debit_count_jun          0
max_credit_amount_jun    0
l

# Encoding for cat data

In [23]:
cat_data=data_train.select_dtypes(include='object').columns
cat_data

Index(['account_type', 'gender', 'loan_enq'], dtype='object')

In [24]:

lab=LabelEncoder()

In [25]:
for col in cat_data:
    data_train[col]=lab.fit_transform(data_train[col])
    

In [26]:
cat_data=['account_type','gender','loan_enq','personal_loan_active', 'vehicle_loan_active','personal_loan_closed','vehicle_loan_closed']

In [27]:
numeric=set(data_train.columns)-set(cat_data)
outlier=data_train[numeric]

In [28]:
def treat_outliers(dataframe):
    cols = list(dataframe)
    for col in cols:
        if col in dataframe.select_dtypes(include=np.number).columns:
            dataframe[col] = winsorize(dataframe[col], limits=[0.05, 0.1],inclusive=(True, True))
    
    return dataframe    


outlier = treat_outliers(outlier)

# Checking for outliers after applying winsorization


C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [29]:
data_train.shape

(15000, 39)

In [30]:
def detect_outliers(dataframe):
    cols = list(dataframe)
    outliers = pd.DataFrame(columns=['Feature','Number of Outliers'])
    
    for column in cols:
        if column in dataframe.select_dtypes(include=np.number).columns:
            # first quartile (Q1)
            q1 = dataframe[column].quantile(0.25) 
            
            # third quartile (Q3)
            q3 = dataframe[column].quantile(0.75)
            
            # IQR
            iqr = q3 - q1
            
            fence_low = q1 - (1.5*iqr)
            fence_high = q3 + (1.5*iqr)
            outliers = outliers.append({'Feature':column,'Number of Outliers':dataframe.loc[(dataframe[column] < fence_low) | (dataframe[column] > fence_high)].shape[0]},ignore_index=True)
    return outliers

detect_outliers(outlier)

,Feature,Number of Outliers
0,debit_count_jun,0
1,credit_count_apr,0
2,cc_cons,0
3,cc_count_apr,0
4,cc_count_jun,0
5,credit_amount_may,0
6,credit_count_may,0
7,debit_count_apr,0
8,dc_count_jun,0
9,investment_1,0


In [38]:
cata=data_train[cat_data]
cata=cata.reset_index().drop('ID',axis=1)

In [39]:
minmax=MinMaxScaler()
minmax.fit(outlier)

MinMaxScaler(copy=True, feature_range=(0, 1))

In [40]:
data_train_scale=pd.DataFrame(minmax.transform(outlier),columns=numeric)
data_train_scale

,debit_count_jun,credit_count_apr,cc_cons,cc_count_apr,cc_count_jun,credit_amount_may,credit_count_may,debit_count_apr,dc_count_jun,investment_1,...,region_code,max_credit_amount_may,dc_cons_apr,dc_count_apr,max_credit_amount_jun,investment_2,max_credit_amount_apr,investment_3,credit_amount_apr,dc_cons_jun
0,0.000000,0.166667,0.047401,0.166667,0.310345,0.712200,0.142857,0.000000,0.166667,0.0,...,0.206704,0.141721,0.664287,0.000000,0.234974,0.0,0.861386,0.0,1.000000,1.000000
1,0.937500,0.500000,1.000000,0.722222,0.965517,0.083075,0.142857,0.655172,0.833333,0.0,...,0.566108,0.561961,1.000000,1.000000,0.906420,0.0,0.770248,0.0,0.263623,1.000000
2,0.718750,0.333333,0.721188,1.000000,0.689655,0.327210,0.428571,0.931034,0.583333,0.0,...,0.588454,0.603470,0.678678,0.714185,0.817908,0.0,0.169929,0.0,0.722785,0.641452
3,1.000000,0.666667,0.292102,0.447978,0.482759,0.209639,0.714286,1.000000,1.000000,0.0,...,0.554935,0.341061,1.000000,0.333333,0.918603,0.0,0.791479,0.0,0.911534,1.000000
4,0.781250,0.666667,0.727591,1.000000,1.000000,0.353523,0.428571,0.137931,1.000000,0.0,...,0.026071,0.900709,0.678678,0.714185,0.576874,0.0,0.483884,0.0,0.487903,1.000000
5,0.250000,0.333333,0.667438,1.000000,1.000000,0.288097,0.285714,0.206897,0.588033,0.0,...,0.873371,0.103524,0.456604,0.500000,0.484129,0.0,0.433202,0.0,0.167911,0.744242
6,0.250000,0.166667,0.209674,0.277778,0.758621,0.883799,0.142857,0.275862,0.000000,0.0,...,0.093110,0.859365,0.678678,0.714185,0.337995,0.0,0.769558,0.0,0.688332,0.618920
7,0.125000,0.166667,0.625740,1.000000,1.000000,0.279721,0.571429,0.482759,0.588033,0.0,...,0.078212,0.832230,0.678678,0.714185,0.399230,0.0,0.748565,0.0,0.034310,0.744242
8,0.343750,0.166667,1.000000,0.222222,0.000000,0.203797,0.285714,1.000000,0.588033,0.0,...,0.150838,0.453307,0.678678,0.714185,0.000000,0.0,0.556690,0.0,0.215059,0.744242
9,0.343750,0.833333,0.253255,0.447978,0.896552,0.194444,0.857143,0.965517,0.083333,0.0,...,0.554935,0.844908,0.678678,0.714185,0.676394,0.0,0.274296,0.0,0.669813,0.590967


In [41]:
merge_data=pd.concat([data_train_scale,cata],axis=1)
merge_data.head()

,debit_count_jun,credit_count_apr,cc_cons,cc_count_apr,cc_count_jun,credit_amount_may,credit_count_may,debit_count_apr,dc_count_jun,investment_1,...,investment_3,credit_amount_apr,dc_cons_jun,account_type,gender,loan_enq,personal_loan_active,vehicle_loan_active,personal_loan_closed,vehicle_loan_closed
0,0.00000,0.166667,0.047401,0.166667,0.310345,0.712200,0.142857,0.000000,0.166667,0.0,...,0.0,1.000000,1.000000,1,0,0,0.0,0.0,0.0,0.0
1,0.93750,0.500000,1.000000,0.722222,0.965517,0.083075,0.142857,0.655172,0.833333,0.0,...,0.0,0.263623,1.000000,0,1,0,0.0,0.0,0.0,0.0
2,0.71875,0.333333,0.721188,1.000000,0.689655,0.327210,0.428571,0.931034,0.583333,0.0,...,0.0,0.722785,0.641452,0,1,1,0.0,0.0,0.0,0.0
3,1.00000,0.666667,0.292102,0.447978,0.482759,0.209639,0.714286,1.000000,1.000000,0.0,...,0.0,0.911534,1.000000,0,1,0,0.0,0.0,0.0,1.0
4,0.78125,0.666667,0.727591,1.000000,1.000000,0.353523,0.428571,0.137931,1.000000,0.0,...,0.0,0.487903,1.000000,0,1,0,0.0,0.0,0.0,0.0


In [45]:
merge_data.columns

Index(['debit_count_jun', 'credit_count_apr', 'cc_cons', 'cc_count_apr',
       'cc_count_jun', 'credit_amount_may', 'credit_count_may',
       'debit_count_apr', 'dc_count_jun', 'investment_1', 'age',
       'debit_amount_jun', 'dc_count_may', 'dc_cons_may', 'debit_amount_may',
       'debit_count_may', 'cc_count_may', 'investment_4', 'credit_count_jun',
       'credit_amount_jun', 'card_lim', 'debit_amount_apr', 'region_code',
       'max_credit_amount_may', 'dc_cons_apr', 'dc_count_apr',
       'max_credit_amount_jun', 'investment_2', 'max_credit_amount_apr',
       'investment_3', 'credit_amount_apr', 'dc_cons_jun', 'account_type',
       'gender', 'loan_enq', 'personal_loan_active', 'vehicle_loan_active',
       'personal_loan_closed', 'vehicle_loan_closed'],
      dtype='object')

# split the data in train test

In [47]:
y=merge_data['cc_cons']
X=merge_data.drop('cc_cons',axis=1)

In [48]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=5)

In [71]:
def model(model,X_train,X_test,y_train,y_test):
    mod=model()
    mod.fit(X_train,y_train)
    y=mod.predict(X_train)
    mse=mean_squared_log_error(y_train,y)
    r2=r2_score(y_train,y)
    return (mse,r2)

In [72]:
model(LinearRegression,X_train,X_test,y_train,y_test)


(0.05019027896969764, 0.003612147457212167)

In [73]:
model(Lasso,X_train,X_test,y_train,y_test)

(0.050355466420237804, 0.0)

In [74]:
model(Ridge,X_train,X_test,y_train,y_test)

(0.05019036177215789, 0.003612725990561616)

In [75]:
y_test

10644    0.577262
13951    0.194609
8082     0.490531
10284    0.567094
13473    0.834553
8801     0.888787
1828     0.385182
2059     0.835468
321      0.104864
8441     0.000000
14896    0.971053
7541     0.674486
2206     0.326859
4424     0.280049
10289    0.281448
3312     0.588884
13704    0.211234
5237     0.724416
9370     1.000000
75       0.048424
11234    0.259389
5848     0.084741
2991     0.082266
4046     0.000000
1043     0.959001
12997    0.877865
12602    0.058431
4025     0.949155
12868    0.464059
11153    0.000000
           ...   
1647     0.256860
3755     0.986495
234      0.852416
5774     0.058216
3124     0.707629
14699    0.611536
5847     0.720758
10627    0.101474
1290     0.136931
3301     0.796460
10752    0.221780
2111     0.146401
9212     0.804961
1466     0.089745
8818     0.143549
8058     0.760626
8635     0.000000
13022    0.241526
14035    0.653180
1585     0.437910
7096     0.909609
9677     0.182449
2838     0.899279
13793    0.810449
1938     1